In [22]:
#install beautifulsoup, import bs4 and pandas
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
#link to site, parse, and find table
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup  = BeautifulSoup(source, "html.parser")
table = soup.find('table')

In [31]:
#df: PostalCode, Borough, Neighborhood
col_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = col_names)

#find and add each postalcode, borough, then neighborhood
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if len(row) == 3:
        df.loc[len(df)] = row

In [32]:
#check table
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [33]:
#remove unassigned
df = df[df['Borough'] != 'Not assigned']
#check table
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [34]:
#df size
df.shape

(103, 3)

In [48]:
#define coordinates for each postal code?
def get_geocode(postal_code):
    # initialize your variable to None
    coords = None
    while(coords is None):
        geo = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        coords = geo.latlng
    lat = coords[0]
    long = coords[1]
    return lat,long
#unsuccessful

In [49]:
#read csv for coordinates
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
#combine tables
df_geo.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
geo_merged = pd.merge(df_geo, df, on = 'Postalcode')
geo_data = geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
